
해당 프로젝트에서는 웹 페이지에서 추출한 텍스트와 이미지 OCR 결과를 FAISS 벡터 저장소에 저장하고, 사용자 질문과 관련된 정보를 검색하여 LLM에 제공함으로써 RAG 시스템을 구현하였습니다.



`bs4`/`BeautifulSoup`: 웹 페이지를 파싱하는 라이브러리

`requests`: HTTP 요청을 보내는 라이브러리

`google.cloud.vision`: Google의 OCR(광학 문자 인식) API

`langchain` 관련 모듈들: 대규모 언어 모델을 활용한 애플리케이션 구축을 위한 프레임워크

`dotenv`: 환경 변수 관리를 위한 라이브러리

---

📌 python-dotenv는 위의 라이브러리들과 함께 설치하면 오류가 나고 따로 빼면 오류가 나지 않고 잘 작동된다. 왜 그럴까?

-> 패키지 의존성 충돌 때문일 가능성이 높다고 함. 여러 패키지를 동시에 설치할 때, 패키지 간의 버전 충돌이 발생할 수 있기 때문에 지금처럼 따로 설치하면 충돌 문제를 줄일 수 있음!

In [ ]:
# 필요한 라이브러리 설치
!pip install langchain-community langchain-chroma langchain-openai bs4 google-cloud-vision unstructured faiss-cpu python-dotenv

# 필요한 라이브러리 임포트
import bs4
import io
import requests
from bs4 import BeautifulSoup
from google.cloud import vision
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document
from dotenv import load_dotenv
import os

In [ ]:
!pip install python-dotenv
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Google Vision API 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/jaeyeon/활동/홍얼홍얼/vision-api-key.json"

# OpenAI 모델 설정 (gpt-4o-mini)
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

### `crawl_web_content`
: 이 함수는 주어진 URL의 웹 페이지에서 내용과 이미지 URL을 추출합니다.
>`requests.get(url)`: 웹 페이지를 요청합니다.

>`BeautifulSoup`: HTML을 파싱하는 객체를 생성합니다.

>`soup.find(class_="css-18vt64m")`: 특정 CSS 클래스를 가진 요소를 찾습니다(여기서는 본문 콘텐츠).

이미지 태그를 찾아 src 속성을 추출하고, 상대 URL을 절대 URL로 변환합니다.


---


### 📌 상대 URL을 절대 URL로 변환하는 이유는?

웹 페이지에서 이미지 태그(<img>)의 src 속성은 두 가지 형태로 존재할 수 있습니다

> **절대 URL**: https://spartacodingclub.kr/images/logo.png와 같이 완전한 웹 주소를 포함합니다.
**상대 URL**: /images/logo.png 또는 images/logo.png와 같이 도메인 없이 경로만 표시합니다.

크롤링 과정에서 상대 URL을 그대로 사용하면 문제가 생깁니다.

따라서,

* /images/logo.png나 images/logo.png는 독립적으로는 유효한 웹 주소가 아닙니다
이미지를 다운로드하거나 Google Vision API로 보내려면 완전한 웹 주소(URL)가 필요합니다. *



In [ ]:
# 웹 크롤링 및 이미지 URL 추출 함수
def crawl_web_content(url):
    response = requests.get(url)
    response.encoding = 'utf-8'  # 인코딩 설정
    soup = BeautifulSoup(response.text, 'html.parser')

    # 특정 클래스의 본문 내용 추출
    content = soup.find(class_="css-18vt64m")

    # 이미지 URL 추출
    image_urls = []
    if content:
        for img in content.find_all('img'):
            if 'src' in img.attrs:
                # 상대 URL을 절대 URL로 변환
                img_src = img['src']
                if not img_src.startswith(('http://', 'https://')):
                    if img_src.startswith('/'):
                        img_src = 'https://spartacodingclub.kr' + img_src
                    else:
                        img_src = 'https://spartacodingclub.kr/' + img_src
                image_urls.append(img_src)

    return content, image_urls

### `extract_text_from_image`
이 함수에서는 Google Vision API를 사용하여 이미지에서 텍스트를 추출합니다(OCR)

>`vision.ImageAnnotatorClient()`: Google Vision API 클라이언트를 생성합니다.

>`requests.get(image_url)`: 이미지를 다운로드합니다.

>`client.text_detection`: 이미지에서 텍스트를 감지하는 API를 호출합니다.

In [ ]:
# Google Vision API를 사용한 OCR 함수
def extract_text_from_image(image_url):
    client = vision.ImageAnnotatorClient()

    try:
        # 이미지를 메모리에 로드
        response = requests.get(image_url)
        image = vision.Image(content=response.content)

        # OCR 실행
        response = client.text_detection(image=image)
        texts = response.text_annotations

        if texts:
            return texts[0].description
        return ""
    except Exception as e:
        print(f"이미지 처리 중 오류 발생: {e}")
        return ""

특정 URL(스파르타 코딩클럽의 ALL-in 공모전 페이지)에서 이미지 URL을 추출합니다.

각 이미지에서 OCR을 사용해 텍스트를 추출하고 저장합니다.

추출된 텍스트는 "이미지 텍스트: "라는 접두사를 붙여 저장하고, 미리보기는 100자로 제한합니다.

---

`UnstructuredURLLoader`를 사용해 **웹 페이지의 전체 내용을 로드**합니다.

로드된 문서의 수와 처음 2개 문서의 내용 미리보기를 출력합니다.

---

### 📌 UnstructuredURLLoader란?
UnstructuredURLLoader는 LangChain 라이브러리에서 제공하는 문서 로더 중 하나로, 웹 페이지의 URL을 입력받아 해당 웹 페이지의 내용을 구조화된 형태로 추출하는 도구입니다.

>- HTML 문서에서 본문 내용, 제목, 메타데이터 등을 자동으로 추출합니다.
- 광고, 네비게이션 메뉴, 푸터 등의 불필요한 요소를 제거하고 실제 콘텐츠만 추출합니다.
- 추출된 콘텐츠는 LangChain의 Document 객체로 변환됩니다.
- 여러 URL을 한 번에 처리할 수 있습니다.

In [ ]:
# 웹 URL 설정
url = "https://spartacodingclub.kr/blog/all-in-challenge_winner"

# 이미지 처리를 위한 크롤링
_, image_urls = crawl_web_content(url)

# 이미지에서 텍스트 추출
image_texts = []
for img_url in image_urls:
    print(f"이미지 처리 중: {img_url}")
    text = extract_text_from_image(img_url)
    if text:
        # 전체 텍스트는 저장하되, 출력은 100자로 제한
        image_texts.append(f"이미지 텍스트: {text}")
        preview_text = text[:100] + "..." if len(text) > 100 else text
        print(f"추출된 텍스트(미리보기):\n{preview_text}\n{'='*50}")
    else:
        print(f"텍스트가 추출되지 않았습니다.\n{'='*50}")

# UnstructuredLoader로 URL 내용 로드
try:
    loader = UnstructuredURLLoader([url])
    docs = loader.load()
    print(f"UnstructuredLoader로 {len(docs)}개 문서 로드됨")

    # 로드된 문서 내용 확인
    print(f"\nUnstructuredLoader 결과:")
    print(f"추출된 문서 수: {len(docs)}")
    for i, doc in enumerate(docs[:2]):  # 처음 2개 문서만 출력
        print(f"문서 {i+1} 미리보기:")
        print(f"{doc.page_content[:200]}...\n")
except Exception as e:
    print(f"UnstructuredLoader 오류: {e}")
    docs = []  # 오류 발생 시 빈 리스트 초기화

이미지 처리 중: https://s3.ap-northeast-2.amazonaws.com/blog.spartacodingclub.kr/1725350620897-%C3%A1%C2%84%C2%89%C3%A1%C2%85%C2%B3%C3%A1%C2%84%C2%8F%C3%A1%C2%85%C2%B3%C3%A1%C2%84%C2%85%C3%A1%C2%85%C2%B5%C3%A1%C2%86%C2%AB%C3%A1%C2%84%C2%89%C3%A1%C2%85%C2%A3%C3%A1%C2%86%C2%BA%202024-09-03%20%C3%A1%C2%84%C2%8B%C3%A1%C2%85%C2%A9%C3%A1%C2%84%C2%92%C3%A1%C2%85%C2%AE%202.14.45.png
추출된 텍스트(미리보기):
기능 소개
한국어
Template
텍스트 인식
맞춤법 검사
번역
DELETE
텍스트 드래그 후 한 번 클릭 시
네이버 사전 팝업 (국어, 영어, 프랑스어, 중국어)
'사랑'의 검색...
이미지 처리 중: https://s3.ap-northeast-2.amazonaws.com/blog.spartacodingclub.kr/1725350707232-%C3%A1%C2%84%C2%89%C3%A1%C2%85%C2%B3%C3%A1%C2%84%C2%8F%C3%A1%C2%85%C2%B3%C3%A1%C2%84%C2%85%C3%A1%C2%85%C2%B5%C3%A1%C2%86%C2%AB%C3%A1%C2%84%C2%89%C3%A1%C2%85%C2%A3%C3%A1%C2%86%C2%BA%202024-09-03%20%C3%A1%C2%84%C2%8B%C3%A1%C2%85%C2%A9%C3%A1%C2%84%C2%92%C3%A1%C2%85%C2%AE%202.12.57.png
추출된 텍스트(미리보기):
Σ
H
전체 v
우리집 히어로즈
우리집 히어로즈
우리집 히어로즈
Prototype
우리집 히어로즈
우리집 히어로즈
최신순 v
전체 v
최신순 v
전체 v
최신순 v
BAR
바잡송
...
이미지 처리 중: https://

이미지에서 추출한 모든 텍스트를 하나의 문자열로 통합합니다.

이 **통합 텍스트**를 **새로운 Document 객체로** 만들어 **기존 문서 리스트에 추가**합니다.
`RecursiveCharacterTextSplitter`를 생성하여 문서를 청크(chunk)로 분할할 준비를 합니다.

> - `chunk_size=1000`: 각 청크의 최대 길이는 1000자
- `chunk_overlap=200`: 인접한 청크 간에 200자의 중복을 허용(문맥 유지를 위함)

---

### 📌 RecursiveCharacterTextSplitter란?
긴 텍스트 문서를 더 작은 청크(chunks)로 분할하는 LangChain의 도구입니다.

> - 텍스트를 의미 있는 단위로 분할합니다(단락, 문장 등의 구분자 기준).
- 재귀적 접근 방식을 사용하여 먼저 가장 큰 구분자(예: 줄바꿈)로 분할을 시도하고, 청크가 여전히 너무 크면 더 작은 구분자(예: 문장, 단어)를 사용합니다.
- chunk_size: 각 청크의 최대 문자 수를 지정합니다(코드에서는 1000자).
- chunk_overlap: 인접한 청크 간에 중복되는 문자 수를 지정합니다(코드에서는 200자). 이는 문맥 연속성을 유지하는 데 도움이 됩니다.

In [ ]:
# 이미지에서 추출한 텍스트를 문서에 추가
if image_texts:
    combined_text = "\n".join(image_texts)
    # 통합 텍스트도 미리보기는 100자로 제한
    preview_text = combined_text[:100] + "..." if len(combined_text) > 100 else combined_text
    print(f"이미지 OCR 통합 텍스트 미리보기:\n{preview_text}\n")
    docs.append(Document(page_content=combined_text, metadata={"source": "image_ocr"}))

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

# 청크 품질 확인을 위한 함수
def analyze_chunk_quality(chunks):
    """청크 품질을 분석하는 함수"""
    quality_metrics = {
        "총 청크 수": len(chunks),
        "평균 청크 길이": sum(len(chunk.page_content) for chunk in chunks) / len(chunks) if chunks else 0,
        "최소 청크 길이": min(len(chunk.page_content) for chunk in chunks) if chunks else 0,
        "최대 청크 길이": max(len(chunk.page_content) for chunk in chunks) if chunks else 0,
        "OCR 텍스트 포함 청크": sum(1 for chunk in chunks if "이미지 텍스트:" in chunk.page_content),
    }

    # 너무 짧은 청크 찾기 (예: 100자 미만)
    short_chunks = [i for i, chunk in enumerate(chunks) if len(chunk.page_content) < 100]

    # 중복 내용이 있는 청크 찾기 (간단한 방법)
    duplicate_content = set()
    potential_duplicates = []

    for i, chunk in enumerate(chunks):
        # 처음 100자를 기준으로 중복 체크 (간단한 예시)
        content_start = chunk.page_content[:100]
        if content_start in duplicate_content:
            potential_duplicates.append(i)
        else:
            duplicate_content.add(content_start)

    quality_metrics["너무 짧은 청크 (인덱스)"] = short_chunks
    quality_metrics["잠재적 중복 청크 (인덱스)"] = potential_duplicates

    return quality_metrics

이미지 OCR 통합 텍스트 미리보기:
이미지 텍스트: 기능 소개
한국어
Template
텍스트 인식
맞춤법 검사
번역
DELETE
텍스트 드래그 후 한 번 클릭 시
네이버 사전 팝업 (국어, 영어, 프랑스어, 중국어)...



In [ ]:
# 문서 분할 후 청크 확인
splits = text_splitter.split_documents(docs)
print(f"\n문서 분할 결과:")
print(f"총 청크 수: {len(splits)}")
print(f"첫 번째 청크 미리보기:\n{splits[0].page_content}\n")
print(f"OCR 텍스트가 포함된 청크 찾기:")
for i, chunk in enumerate(splits):
    if "이미지 텍스트:" in chunk.page_content:
        print(f"청크 {i+1}에 OCR 텍스트 포함됨:\n{chunk.page_content[:200]}...\n")
        break


문서 분할 결과:
총 청크 수: 15
첫 번째 청크 미리보기:
포인트

로딩중

쿠폰

내 강의실

국비 신청 내역

수강권

증명서

숙제 피드백

계정

로그아웃

1725353737651-%C3%A1%C2%84%C2%8F%C3%A1%C2%85%C2%A9%C3%A1%C2%84%C2%83%C3%A1%C2%85%C2%B5%C3%A1%C2%86%C2%BC%C3%A1%C2%84%C2%80%C3%A1%C2%85%C2%A9%C3%A1%C2%86%C2%BC%C3%A1%C2%84%C2%86%C3%A1%C2%85%C2%A9%C3%A1%C2%84%C2%8C%C3%A1%C2%85%C2%A5%C3%A1%C2%86%C2%AB+%C3%A1%C2%84%C2%89%C3%A1%C2%85%C2%AE%C3%A1%C2%84%C2%89%C3%A1%C2%85%C2%A1%C3%A1%C2%86%C2%BC%C3%A1%C2%84%C2%8C%C3%A1%C2%85%C2%A1%C3%A1%C2%86%C2%A8.png

스파르타 소식

'AII-in 코딩 공모전’ 수상작을 소개합니다

조회수 756·6분 분량

2024. 9. 3.

코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.

<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.

🏆 대상

[Lexi Note] 언어공부 필기 웹 서비스

서비스 제작자: 다나와(김다애, 박나경)

OCR 텍스트가 포함된 청크 찾기:
청크 10에 OCR 텍스트 포함됨:
이미지 텍스트: 기능 소개
한국어
Template
텍스트 

OpenAI의 임베딩 모델을 사용하여 각 텍스트 청크를 벡터로 변환합니다.

FAISS 벡터 저장소를 생성하여 이 벡터들을 저장합니다.

벡터 저장소를 검색기(retriever)로 변환하고, 검색 시 상위 15개(k=15) 결과를 반환하도록 설정합니다.


---
### 📎 FAISS란?
FAISS(Facebook AI Similarity Search)는 Facebook(Meta)에서 개발한 고효율 유사성 검색 및 클러스터링 라이브러리입니다. 대량의 벡터에서 빠르게 유사한 항목을 검색할 수 있게 해주는 기술입니다.

출처:https://velog.io/@injokim/FAISS%EB%9E%80

- 특징
>**고차원 벡터**에 대한 *빠른 유사도 검색*과 *다양한 인덱싱 방법*을 지원하여 검색 속도와 정확도 간의 트레이드오프를 조절할 수 있습니다.

해당 프로젝트에서는 **텍스트 청크의 임베딩 벡터를 저장하고, 질문과 가장 유사한 벡터를 찾아 관련 문서를 검색하는 데 사용**되었습니다.

---
### ⁉️ **FAISS 벡터 저장소가 필요한 이유**

1. **효율적인 유사도 검색**

FAISS는 대량의 고차원 벡터에서 빠르게 유사한 항목을 검색할 수 있게 해줍니다.

2. **의미 기반 검색**

텍스트를 벡터로 변환하면 단순한 키워드 일치가 아닌 **의미적 유사성을 기반으로 검색**할 수 있습니다. 예를 들어, "자동차"와 "차량"은 다른 단어지만 의미적으로 유사하므로 벡터 공간에서 가까이 위치합니다.

3. **대규모 데이터 처리**

FAISS는 수백만 개의 벡터를 효율적으로 저장하고 검색할 수 있도록 최적화되어 있습니다.

4. **RAG 시스템의 핵심 구성 요소**
질문에 **관련된 정보를 정확히 검색**하여 LLM에 제공하기 위해 필요합니다.

---
### 📎 벡터 저장소를 검색기(retriever)로 변환하다?

벡터 저장소를 검색기(retriever)로 변환한다는 것은 벡터 데이터베이스에 대한 접근 방식을 표준화하는 추상화 과정 것.

- 변환 과정

as_retriever() 메서드를 호출하면 FAISS 벡터 저장소를 LangChain의 표준 검색기 인터페이스로 변환합니다.
- 검색 매개변수 설정

코드에서 search_kwargs={"k": 15}는 검색 시 가장 유사한 상위 15개의 문서를 반환하도록 지정합니다.

이 변환을 통해 다양한 벡터 저장소 기술(FAISS, Chroma, Pinecone 등)을 동일한 방식으로 사용할 수 있고, **검색 로직을 일관되게 적용**할 수 있습니다.

>간단히 말해, 벡터 저장소는 데이터를 저장하는 장소이고, 검색기는 그 데이터에 접근하여 관련 정보를 검색하는 인터페이스입니다. 이렇게 함으로써 **코드의 다른 부분에서는 구체적인 저장 방식에 대해 신경 쓰지 않고 일관된 방식으로 정보를 검색**할 수 있습니다.

In [ ]:
# FAISS 벡터 저장소 생성
embeddings = OpenAIEmbeddings(api_key=api_key)
vectorstore = FAISS.from_documents(splits, embeddings)

# 검색기 생성
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 15}
)

---

사용자 질문("ALL-in 코딩 공모전 수상작들을 요약해줘")을 설정합니다.

이 질문과 관련된 문서를 검색합니다.

LLM에 전달할 사용자 정의 프롬프트를 작성합니다.

In [ ]:
# 사용자 질문에 대한 문서 검색
question = "ALL-in 코딩 공모전 수상작들을 요약해줘"
retrieved_docs = retriever.get_relevant_documents(question)

# 검색된 문서와 질문을 LLM에 전달
custom_prompt = """당신은 RAG(검색 증강 생성) 시스템입니다.
다음은 ALL-in 코딩 공모전에 대한 정보입니다:
{context}
요청: {question}
중요 지침:
1. 위 정보를 바탕으로 ALL-in 코딩 공모전의 모든 수상작들을 모르는 사람들도 잘 이해하기 쉽게 요약해주세요.
2. 각 수상작의 이름, 등급(대상/우수상/장려상), 개발자, 주요 기능을 포함해야 합니다.
3. 어떤 수상작도 빠뜨리지 말고 모두 포함해주세요.
4. 이미지 OCR로 추출된 정보도 함께 고려해주세요.
답변:"""

검색된 문서들을 하나의 문자열로 포맷팅합니다.

프롬프트에 컨텍스트와 질문을 삽입합니다.

---

In [ ]:
# 검색된 문서를 포맷팅하여 컨텍스트로 제공
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

user_prompt = {"context": format_docs(retrieved_docs), "question": question}
messages = [{"role": "user", "content": custom_prompt.format(**user_prompt)}]
response = llm.invoke(messages)
print(response.content)

### ALL-in 코딩 공모전 수상작 요약

1. **대상 - Lexi Note**
   - **개발자**: 다나와(김다애, 박나경)
   - **주요 기능**: 언어 공부를 위한 필기 웹 서비스로, 단어를 드래그하여 사전으로 연동하고, 번역기가 통합되어 있어 번역과 학습을 동시에 수행할 수 있습니다. 할 일 목록과 스케줄 템플릿을 제공하여 효율적인 학습을 지원합니다.

2. **우수상 - 우리집 히어로즈**
   - **개발자**: 인트(배정연, 한지수)
   - **주요 기능**: 벌레 퇴치 영웅과 자취생을 매칭하는 서비스입니다. 사용자는 벌레 문제를 해결할 수 있는 "히어로"를 요청하며, 안전한 환경에서 신원을 보장받고 실시간 알림을 통해 매칭 정보를 받아볼 수 있습니다.

3. **우수상 - 에코 클래스룸**
   - **개발자**: This is 스파게티!!!(박지성, 김서원, 박범수)
   - **주요 기능**: 교수가 학생들의 이해도를 실시간으로 파악할 수 있도록 돕는 수업 소통 서비스입니다. 학생은 익명으로 질문이나 의견을 제출할 수 있으며, 교수는 수업 속도를 조절할 수 있는 퀴즈 및 평가 기능을 갖추고 있습니다.

4. **입선 - Crewing**
   - **개발자**: 동학대학운동(김민아, 임경진, 신은혜, 고수)
   - **주요 기능**: 대학생들이 적절한 연합 동아리를 찾을 수 있도록 지원하는 플랫폼입니다. 사용자 맞춤형 동아리 추천 및 리크루팅 과정 관리 기능을 제공합니다.

5. **입선 - 학교생활 매니저**
   - **개발자**: 아이칼F4(조민제, 이민기, 강건, 박근우)
   - **주요 기능**: 학교 생활을 관리할 수 있는 앱으로, 일정 관리, 성적 예측, 학점 계산 등 다양한 기능을 통해 대학생들이 효율적으로 학업을 관리할 수 있도록 돕습니다.

6. **입선 - BLOTIE**
   - **개발자**: 블로티(이은주, 한명수, 황준영)
   - **주요 기능**: 외국인 학생과 한국인 학생을 매칭하는 플랫폼으로

### 성공!